In [15]:
import json
import argparse
import glob
import sys
sys.path.append("../")
from tool import finqa_equal
from typing import Dict, Any


In [65]:
name = 'convfinqa_direct_gpt-4_04_10_13_02.jsonl'

In [66]:
exception = []
correct, wrong, missing = 0, 0, 0
with open(name) as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if 'demonstration' in data or 'prompt' in data:
            continue
        else:
            # Select the prediction field.
            if 'executed' in data:
                prediction = data['executed']
            else:
                prediction = data['prediction']
                if False:
                    # For multi-choise question, pick A option
                    if prediction == 'None':
                        prediction = 'A'
            # Calculate the evaluation score
            pred = finqa_equal(prediction, data['answer'], True, True)
            if prediction is None:
                missing += 1
            elif pred:
                correct += 1
            else:
                exception.append(data)
                wrong += 1

print('file name: ', name)
print('num of examples: ', correct + wrong + missing)
print('num of missing returns', missing)
print('accuracy: ', correct / (correct + wrong + missing))

file name:  convfinqa_direct_gpt-4_04_10_13_02.jsonl
num of examples:  421
num of missing returns 0
accuracy:  0.6318289786223278


In [67]:
import pandas as pd

In [68]:
incorrect = pd.DataFrame(exception)

In [69]:
prompt_4shot = """Read the following text and table, and then answer the last question in a series of questions:
- | shares available for awards | shares subject to outstanding awards
2009 global incentive plan | 2322450 | 2530454
2004 stock incentive plan | - | 5923147
Questions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?
Question: what proportion does this represent?
#Python
shares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454
shares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147
total_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan
proportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards
ans = proportion


Read the following text and table, and then answer the last question in a series of questions:
compensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . 
Questions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?
Question: what is the average for three years?
#Python
compensation_expense_2015 = 43
compensation_expense_2014 = 34
compensation_expense_2013 = 44
total_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013
average_for_three_years = total_compensation_expense / 3
ans = average_for_three_years


Read the following text and table, and then answer the last question in a series of questions:
the net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . 
Questions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?
Question: what was the percent change?
#Python
net_loss_on_disposal_of_assets_2005 = 344000
net_loss_on_disposal_of_assets_2004 = 43000
net_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004
percent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004
ans = percent_change


Read the following text and table, and then answer the last question in a series of questions:
location | operations conducted | approximatesquare feet | leaseexpirationdates
dublin ireland | global supply chain distribution and administration offices | 160000 | owned
athlone ireland | commercial research and development manufacturing | 80000 | owned
bogart georgia | commercial research and development manufacturing | 70000 | owned
smithfield rhode island | commercial research and development manufacturing | 67000 | owned
Questions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?
Question: what is the total sum of square feet owned?
#Python
square_feet_owned = [160000, 80000, 70000, 67000]
total_square_feet_owned = sum(square_feet_owned)
ans = total_square_feet_owned
"""

In [70]:
prompt_msg_4shot = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\n- | shares available for awards | shares subject to outstanding awards\n2009 global incentive plan | 2322450 | 2530454\n2004 stock incentive plan | - | 5923147\nQuestions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?\nQuestion: what proportion does this represent?"},{"role":"assistant","content":"#Python\nshares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454\nshares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147\ntotal_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan\nproportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards\nans = proportion"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\ncompensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . \nQuestions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?\nQuestion: what is the average for three years?"},{"role":"assistant","content":"#Python\ncompensation_expense_2015 = 43\ncompensation_expense_2014 = 34\ncompensation_expense_2013 = 44\ntotal_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013\naverage_for_three_years = total_compensation_expense / 3\nans = average_for_three_years\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nthe net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . \nQuestions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?\nQuestion: what was the percent change?"},{"role":"assistant","content":"#Python\nnet_loss_on_disposal_of_assets_2005 = 344000\nnet_loss_on_disposal_of_assets_2004 = 43000\nnet_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004\npercent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004\nans = percent_change\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nlocation | operations conducted | approximatesquare feet | leaseexpirationdates\ndublin ireland | global supply chain distribution and administration offices | 160000 | owned\nathlone ireland | commercial research and development manufacturing | 80000 | owned\nbogart georgia | commercial research and development manufacturing | 70000 | owned\nsmithfield rhode island | commercial research and development manufacturing | 67000 | owned\nQuestions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?\nQuestion: what is the total sum of square feet owned?"},{"role":"assistant","content":"#Python\nsquare_feet_owned = [160000, 80000, 70000, 67000]\ntotal_square_feet_owned = sum(square_feet_owned)\nans = total_square_feet_owned"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:  \nentergy new orleans estimates that approximately 141000 electric customers and 93000 gas customers have returned since hurricane katrina and are taking service as of december 31 , 2008 , compared to approximately 132000 electric customers and 86000 gas customers as of december 31 , 2007 .  \nQuestions: what was the number of gas customers in 2008? and what was it in 2007? what was, then, the change in that number over the year?  \nQuestion: and how much does this change represent in relation to the 2007 number of customers, in percentage?"},{"role":"assistant","content":"#Python\ngas_customers_2008 = 93000\ngas_customers_2007 = 86000\nchange_in_gas_customers = gas_customers_2008 - gas_customers_2007\npercent_change = (change_in_gas_customers / gas_customers_2007) / gas_customers_2007\nans = percent_change"}]

In [71]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    prompt += '\n'
    prompt += 'Answer:'
    return prompt

In [72]:
def create_reader_request_processed_chat(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    return prompt

In [73]:
full_prompt = prompt_4shot + "\n\n"+ create_reader_request_processed(incorrect.iloc[1,])

In [74]:
incorrect['prompt'] = incorrect.apply(lambda x:  prompt_4shot + "\n\n"+ create_reader_request_processed(x), axis=1)
incorrect['chatprompt'] = incorrect.apply(lambda x:  prompt_msg_4shot + [{"role":"user","content":create_reader_request_processed_chat(x)}], axis=1)

In [77]:
incorrect.columns

Index(['questions', 'answer', 'text', 'table', 'golden_text', 'golden_table',
       'id', 'programs', 'generated', 'executed', 'prompt', 'chatprompt'],
      dtype='object')

In [79]:
incorrect[['answer','programs','generated', 'executed']]

,answer,programs,generated,executed
0,0.375,"[ans = 15 - 19, x0 = 15 - 19; ans = x0 / 19, a...",[#Python\nunamortized_debt_issuance_costs_seni...,142.402546
1,14.0,"[ans = 12000000 / 3000000, ans = 37.8, ans = 2...",[#Python\ngallons_hedged_2017 = 12000000\ngall...,-9.708738
2,18770.0,"[ans = 129.05 - 110.28, x0 = 129.05 - 110.28; ...",[#Python\nprice_2015 = 110.28\nprice_2016 = 10...,-740.0
3,2525.0,"[ans = 2.2 * 1000, ans = 325, x0 = 2.2 * 1000;...",[#Python\ntotal_expense_repairs_maintenance_20...,2525000000.0
4,1.25,"[ans = 4.7 - .3, x0 = 4.7 - .3; ans = 4 - .7, ...",[#Python\nforeign_exchange_products_variance_s...,2525000000.0
...,...,...,...,...
150,0.0,"[ans = 2.6 - 2.6, x0 = 2.6 - 2.6; ans = x0 / 2.6]",[#Python\nfreight_revenue_2013 = 2.6\nfreight_...,86.5
151,0.07039,"[ans = 822690, ans = 768588, ans = 822690 - 76...",[#Python\nlong_term_debt_maturities_2018 = 822...,6.576232
152,0.17792,"[ans = 5829, ans = 148527, ans = 5829 + 148527...",[#Python\nliabilities_assumed = -5829\npositiv...,0.164482
153,80.0,"[ans = 36 / 45, x0 = 36 / 45; ans = x0 * 100]",[I apologize for the confusion in my previous ...,I apologize for the confusion in my previous r...


In [95]:
print(incorrect['generated'][2][0])

#Python
price_2015 = 110.28
price_2016 = 109.54
change_in_price = price_2016 - price_2015
quotient = 100000 / 100
product = change_in_price * quotient
ans = product


In [97]:
price_2015 = 110.28
price_2016 = 109.54
change_in_price = price_2016 - price_2015
quotient = 100000 / 100
product = change_in_price * quotient
ans = product

In [105]:
print(incorrect['golden_table'][2])

- | 12/28/2013 | 1/3/2015 | 1/2/2016 | 12/31/2016 | 12/30/2017 | 12/29/2018
s&p 500 | 100.00 | 110.28 | 109.54 | 129.05 | 157.22 | 150.33



In [99]:
print(incorrect['programs'][2])

['ans = 129.05 - 110.28', 'x0 = 129.05 - 110.28; ans = 100000 / 100', 'x0 = 129.05 - 110.28; x1 = 100000 / 100; ans = x1 * x0']
